In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#???
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

#Импорт таблиц номинального режима и частичной нагрузки
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
gas_streams    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streams  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#???
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)


######################################################################
#Связвка высокого давления
start_time = time.time()
it=50
for j in range(it):
    for i in range(it):
        PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
        water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
        IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
        Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
        Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
        print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
        if  abs((Qgas-Qwat)/Qgas) < calctolerance:
            break;
    EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
    water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
    
    Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
    Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
    print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
    if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
        break;
print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print("ВД:--- %s сек. ---" % round((time.time() - start_time)))
######################################################################



######################################################################
#Связка низкого давления
it=50
for j in range(it):
    #Связка ППНД+ИНД
    for i in range(it):
        #Расчёт ППНД
        PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
        #Расчёт ИНД
        IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
        #Переопределение расхода пара
        water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
        Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
        Qwat = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
        if  abs((Qgas-Qwat)/Qgas) < calctolerance:
            break;
    #ПЭН
    water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
    PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
    water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
    
    
    #Расчёт расхода в ГПК
    water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
    water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
    #Расчёт ГПК
    GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
    water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]#!!!!!!
    
    
    Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
    Qwat1 = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['X-GPK','H'])
    print('dQ/Q ППНД+ИНД+ГПК',((Qgas1-Qwat1)/Qgas1))
    if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
        break;
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("НД:--- %s сек. ---" % round((time.time() - start_time)))
######################################################################






dQ/Q ПЕВД+ИВД 0.044547957328806266
dQ/Q ПЕВД+ИВД -0.014681196198045561
dQ/Q ПЕВД+ИВД 0.0046193697536517355
dQ/Q ПЕВД+ИВД -0.0014752527289871073
dQ/Q ПЕВД+ИВД 0.0004689212957389427
dQ/Q ПЕВД+ИВД+ЭВД 0.0014076862234722615
dQ/Q ПЕВД+ИВД 0.0004444792173603504
dQ/Q ПЕВД+ИВД+ЭВД 0.0011594005930934236
dQ/Q ПЕВД+ИВД 0.0003369676796989908
dQ/Q ПЕВД+ИВД+ЭВД 0.0008730215250036114
Степень сухости пара в ЭВД:  -9999990.0
ВД:--- 5 сек. ---
dQ/Q ППНД+ИНД -0.00278412991624113
dQ/Q ППНД+ИНД 0.0022061794889544073
dQ/Q ППНД+ИНД -5.388797086024003e-05
dQ/Q ППНД+ИНД+ГПК -0.015633799169756855
dQ/Q ППНД+ИНД -0.0017283609582632992
dQ/Q ППНД+ИНД 4.080443365814557e-05
dQ/Q ППНД+ИНД+ГПК -0.020828493484552082
dQ/Q ППНД+ИНД -0.0010324511080194532
dQ/Q ППНД+ИНД 2.3594749015940046e-05
dQ/Q ППНД+ИНД+ГПК -0.02400269547368153
dQ/Q ППНД+ИНД -0.0006219229816615128
dQ/Q ППНД+ИНД+ГПК -0.026260871886118257
dQ/Q ППНД+ИНД -0.00020750145295602966
dQ/Q ППНД+ИНД+ГПК -0.027212565031564755
dQ/Q ППНД+ИНД -0.00023079485597063653
dQ/

KeyboardInterrupt: 

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,936.681566,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,457.281687,0.1,862.085280,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,308.509827,0.1,696.912954,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,241.378260,0.1,624.221649,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,235.960781,0.1,618.401133,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,171.186311,0.1,549.282711,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,113.368384,0.1,488.231148,516.2,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
water_streams

,T,P,H,G
PEVD-DROS,498.473347,8.026691,3395.387386,59.370471
IVD-PEVD,297.938037,8.341200,2753.507839,59.370471
EVD-IVD,297.911601,8.341200,1333.353901,59.370471
PEN-EVD,165.671153,8.341200,704.502121,59.370471
BND-PEN,164.512241,0.692500,695.109778,59.370471
PPND-TURB,242.810388,0.685457,2939.225157,16.811977
IND-PPND,164.512241,0.692500,2762.297432,16.811977
GPK-IND,156.713048,0.692500,661.283925,76.182448
X-GPK,60.000000,0.692500,251.745121,76.182448
GPK-RECIRC,NaN,0.692500,NaN,11.980000
